# Resource breaks

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/PyJobShop/PyJobShop/blob/main/examples/breaks.ipynb)

> If you're using this notebook in Google Colab, be sure to install PyJobShop first by executing ```pip install pyjobshop``` in a cell.

This notebook demonstrates how to model and handle breaks (interruptions) in scheduling problems using PyJobShop.

We'll cover the following topics:
- A basic example
- Show how to allow tasks to be interrupted by breaks
- Show how breaks work with multiple modes
- Show an example where breaks can be used to model operator availability

## Introduction

In real-world scheduling, machines and resources often have planned breaks or unavailable periods:
- Maintenance windows
- Shift changes
- Lunch breaks
- Planned downtime

PyJobShop allows you to model these breaks and ensures tasks are scheduled around them, or interrupted and resumed after breaks.

## Basic example

Let's create a simple scheduling problem with machine breaks.

In [ ]:
import random

from pyjobshop import Model

random.seed(42)

model = Model()

We create two machines, both with breaks of [0, 8] (outside working hours) and [12, 13] (lunch break). 

In [ ]:
machine1 = model.add_machine(breaks=[(0, 8), (12, 13)], name="Machine 1")
machine2 = model.add_machine(breaks=[(0, 8), (12, 13)], name="Machine 2")

We randomly generate some tasks, with processing times between 1 and 3 hours:

In [ ]:
job = model.add_job(due_date=17)  # represents overtime work after 17h

for _ in range(10):
    task = model.add_task(job)
    processing_time = random.randint(1, 3)
    model.add_mode(task, machine1, processing_time)
    model.add_mode(task, machine2, processing_time)

model.set_objective(weight_max_tardiness=1);

In [ ]:
print(model.summary())

In [ ]:
result = model.solve(display=False)
print(result)

In [ ]:
from pyjobshop.plot import plot_machine_gantt

plot_machine_gantt(result.best, model.data())

## Job shop with operator breaks

In many job shops, machines require operator supervision during setup but can operate independently during processing. This example shows how to model such scenarios using synchronized tasks with operator breaks.

In [ ]:
model = Model()
job = model.add_job()

# Create the machine resource and the operator resource (with breaks).
machine = model.add_machine(name="machine")
operator = model.add_machine(breaks=[(0, 8), (17, 24)], name="operator")

# The processing task is the one that will be processed by the machine.
processing_task = model.add_task(job, name="processing")
duration = 15
model.add_mode(processing_task, machine, duration=duration)

# The setup task is the one that will be processed by the operator.
setup_task = model.add_task(job, name="setup")
duration = 1
model.add_mode(setup_task, operator, duration=duration)

# The machine and operator task should start at the same time. This means
# that we can only start the setup when an operator is available, but
# the machine can continue processing without the operator. Note that
# using the start-before-start constraint twice with reverse arguments
# is the same as a start-at-start constraint.
model.add_start_before_start(processing_task, setup_task)
model.add_start_before_start(setup_task, processing_task)

# The plot shows that the tasks can only start when the operator is available,
# which is at time 8. The processing task starts at time 8 and ends at time 23,
# not requiring the operator's attendance during processing.
result = model.solve(display=False)
plot_machine_gantt(result.best, model.data(), plot_labels=True)

Key points in this example:

- The operator has breaks from 0-8 and 17-24, so they're only available during working hours (8-17)
- Both tasks must start simultaneously using bidirectional start-before-start constraints
- The machine can continue processing after the operator finishes setup
- The Gantt chart shows tasks starting at time 8 when the operator becomes available